In [2]:
import sys
sys.version_info



sys.version_info(major=3, minor=12, micro=12, releaselevel='final', serial=0)

In [2]:
from pyspark.sql import SparkSession, functions as F
import pandas

In [8]:
spark = (
    SparkSession
    .builder
    # .master('dev-java.home.lan')
    .master('local[4]')
    .appName('mySpark')
    .getOrCreate()
)


In [4]:
# spark.stop()


In [9]:
spark.catalog.currentCatalog()

'spark_catalog'

In [27]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, TimestampType

sm = StructType(fields = [
    StructField("col_1", StringType()),
    StructField("col_2", StringType()),
    StructField("id", IntegerType()),
    StructField("dt_ts", TimestampType()),
    StructField("dt", DateType()),
    StructField("dt1", StringType())
])
df = spark.read.csv("1.csv", schema=sm, sep=';')
df.printSchema()
df.show(5)
#df_ = df.withColumn("dtn", F.to_timestamp("dt1", when("dd-MM-yyyy'T'HH:mm:ss", df["dt1"].contains("T"))))
# df_ = df.withColumn(
#     "dtn1",
#     F.when(df["dt1"].contains("T"), F.to_timestamp("dt1", "dd-MM-yyyy'T'HH:mm:ss"))
#     .otherwise(F.to_timestamp("dt1", "dd-MM-yyyy HH:mm:ss"))
# )
df_ = df.withColumn(
    "dtn1",
    F.date_format(
    F.when(df.dt1.contains("T"), F.to_timestamp("dt1", "dd-MM-yyyy'T'HH:mm:ss"))
    .otherwise(F.to_timestamp("dt1", "dd-MM-yyyy HH:mm:ss"))
        , "dd--MM--yyyy HH:mm"
    )
)
#df_ = df.withColumn("dtn", F.to_timestamp("dt1", "dd-MM-yyyy HH:mm:ss"))
df_.printSchema()
df_.show(5, truncate=False)
#df.select('*', F.regexp_replace('dt1', r'[^\d:-]', ' ')).show()



root
 |-- col_1: string (nullable = true)
 |-- col_2: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- dt_ts: timestamp (nullable = true)
 |-- dt: date (nullable = true)
 |-- dt1: string (nullable = true)

+-----+-----+---+-------------------+----------+-------------------+
|col_1|col_2| id|              dt_ts|        dt|                dt1|
+-----+-----+---+-------------------+----------+-------------------+
|   s1| s2-1|  1|2025-12-25 13:00:56|      NULL|25-12-2025T13:01:56|
|   s2| s2-2|  2|2025-11-30 01:19:45|      NULL|               NULL|
|   s3| s2-3|  3|2025-12-26 00:00:00|2025-11-29|25-11-2021 14:10:56|
|   s4| s2-4|  4|               NULL|      NULL|               NULL|
+-----+-----+---+-------------------+----------+-------------------+

root
 |-- col_1: string (nullable = true)
 |-- col_2: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- dt_ts: timestamp (nullable = true)
 |-- dt: date (nullable = true)
 |-- dt1: string (nullable = true)
 |--

In [29]:
df_ = df.withColumn(
    "dtn1",
    F.date_format(
        F.when(df.dt1.contains("T"), F.to_timestamp("dt1", "dd-MM-yyyy'T'HH:mm:ss"))
                .otherwise(F.to_timestamp("dt1", "dd-MM-yyyy HH:mm:ss"))
        , "dd--MM--yyyy HH:mm"
    )
)
#df_ = df.withColumn("dtn", F.to_timestamp("dt1", "dd-MM-yyyy HH:mm:ss"))
# df_.printSchema()
df_.show(5, truncate=False)


+-----+-----+---+-------------------+----------+-------------------+------------------+
|col_1|col_2|id |dt_ts              |dt        |dt1                |dtn1              |
+-----+-----+---+-------------------+----------+-------------------+------------------+
|s1   |s2-1 |1  |2025-12-25 13:00:56|NULL      |25-12-2025T13:01:56|25--12--2025 13:01|
|s2   |s2-2 |2  |2025-11-30 01:19:45|NULL      |NULL               |NULL              |
|s3   |s2-3 |3  |2025-12-26 00:00:00|2025-11-29|25-11-2021 14:10:56|25--11--2021 14:10|
|s4   |s2-4 |4  |NULL               |NULL      |NULL               |NULL              |
+-----+-----+---+-------------------+----------+-------------------+------------------+



In [30]:
df2 = spark.read.json("2.json", sm, multiLine=True)
df2.printSchema()
df2.show()

root
 |-- col_1: string (nullable = true)
 |-- col_2: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- dt_ts: timestamp (nullable = true)
 |-- dt: date (nullable = true)
 |-- dt1: string (nullable = true)

+-----+-----+----+-----+----+----+
|col_1|col_2|  id|dt_ts|  dt| dt1|
+-----+-----+----+-----+----+----+
| NULL| NULL|NULL| NULL|NULL|NULL|
+-----+-----+----+-----+----+----+



In [31]:
df3 = spark.read.json("3.json", sm, multiLine=True)
df3.printSchema()
df3.show()

root
 |-- col_1: string (nullable = true)
 |-- col_2: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- dt_ts: timestamp (nullable = true)
 |-- dt: date (nullable = true)
 |-- dt1: string (nullable = true)

+-----+-----+----+-----+----+----+
|col_1|col_2|  id|dt_ts|  dt| dt1|
+-----+-----+----+-----+----+----+
| s1-1| s1-2|NULL| NULL|NULL|NULL|
| s2-1| s2-2|NULL| NULL|NULL|NULL|
+-----+-----+----+-----+----+----+



In [34]:
df.createOrReplaceTempView("view1")
df5 = spark.sql("select * from view1")
df5.show()
df4 = spark.sql("select * from view1 where dt_ts >= dt")
df4.show()



+-----+-----+---+-------------------+----------+-------------------+
|col_1|col_2| id|              dt_ts|        dt|                dt1|
+-----+-----+---+-------------------+----------+-------------------+
|   s1| s2-1|  1|2025-12-25 13:00:56|      NULL|25-12-2025T13:01:56|
|   s2| s2-2|  2|2025-11-30 01:19:45|      NULL|               NULL|
|   s3| s2-3|  3|2025-12-26 00:00:00|2025-11-29|25-11-2021 14:10:56|
|   s4| s2-4|  4|               NULL|      NULL|               NULL|
+-----+-----+---+-------------------+----------+-------------------+

+-----+-----+---+-------------------+----------+-------------------+
|col_1|col_2| id|              dt_ts|        dt|                dt1|
+-----+-----+---+-------------------+----------+-------------------+
|   s3| s2-3|  3|2025-12-26 00:00:00|2025-11-29|25-11-2021 14:10:56|
+-----+-----+---+-------------------+----------+-------------------+

